In [13]:
import os
import pandas as pd

import classifier
import inputs

# py-slippi
from slippi import Game

In [2]:
PATH='dataset/demo/inference'

In [16]:
file = os.path.join(PATH, os.listdir(PATH)[0])
game = Game(file)
game

Game(
    end=End(
        lras_initiator=None,
        method=1:TIME),
    frames=[...](18124),
    metadata=Metadata(
        console_name=None,
        date=2024-04-29 23:46:26+00:00,
        duration=18124,
        platform='dolphin':DOLPHIN,
        players=(
            Player(
                characters={9:PEACH: 18124},
                netplay=None),
            None,
            None,
            Player(
                characters={1:FOX: 18124},
                netplay=None))),
    start=Start(
        is_frozen_ps=False,
        is_pal=False,
        is_teams=False,
        players=(
            Player(
                character=12:PEACH,
                costume=0,
                stocks=4,
                tag=,
                team=None,
                type=0:HUMAN,
                ucf=UCF(
                    dash_back=1:UCF,
                    shield_drop=1:UCF)),
            None,
            None,
            Player(
                character=2:FOX,
                co